In [2]:
import pandas as pd

# data of file 2016

#### reading data and rename shifted columns

In [25]:
df = pd.read_csv(r"D:\mohamed\archive\ks-projects-201612(مشروع من النت).csv")

df.rename(columns ={'ID ' : 'id' , 'Unnamed: 14' : 'shitted_col2' , 'Unnamed: 13' : 'shitted_col1'
                   , 'Unnamed: 15' : 'shitted_col3' , 'Unnamed: 16' : 'shitted_col4'}  , inplace=True)
df.shape

C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3030458219.py:1: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\mohamed\archive\ks-projects-201612(مشروع من النت).csv")


(323750, 17)

#### making subset of unshifted data 

In [4]:
cleaned_df =  df[df['shitted_col1'].isnull()]

cleaned_df.loc[cleaned_df['usd pledged '].isnull(), ['country ', 'usd pledged ']] = ['NO', 0]

cleaned_df = cleaned_df.drop(['shitted_col2','shitted_col1','shitted_col3','shitted_col4'],axis = 1)


#### function for updatting shifted data 

In [5]:
def updatting_date(df) :
    shiffted_df = df[df['shitted_col1'].notnull()]
    
    shiffted_df.columns = map(str.upper, shiffted_df.columns)
    columns = list(shiffted_df.columns)
    cols_merged = list(map(str.lower, columns[:-1]))
    columns.remove('CATEGORY ')
    
    return shiffted_df , columns , cols_merged

shiffted_df , columns , cols_merged = updatting_date(df)

#### function to merge columns which separated wrongly

In [6]:
def merge_function(df , pos , list1 ) :
    
    new_order = list(df.columns)
        
    df.loc[:, 'blank'] = df[list1[0]]

    for col in list1[1: ]:
        df.loc[: ,'blank'] += ' ' + df[col].astype(str)
    
    [df.pop(x) for x in list1]
    [new_order.remove(x) for x in list1]

    new_order.insert(pos-1 , 'blank')
    
    df = df[new_order]
    
    df = df.rename(columns={'blank' : 'NAME '})
    
    df['NAME '] = df['NAME '].str.strip()
    
        
    return df

shiffted_df = merge_function(shiffted_df , 2 , ['NAME ' , 'CATEGORY '])



C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3305884801.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'blank'] = df[list1[0]]


#### function to shift columns left

In [7]:
def shifting(df , cols_merged , columns):
    for i in range(len(cols_merged)):
        df = df.rename(columns = {columns[i] : cols_merged[i] })
    return df
        
shiffted_df = shifting(shiffted_df , cols_merged , columns)

#### function to append the clean data 

In [8]:
def append_frames(cleaned_df , frame):
    if 'shitted_col1' in frame.columns:
        frame = frame.loc[frame['shitted_col1'].isnull()]
    cleaned_df = pd.concat([frame , cleaned_df])
    return cleaned_df

frame = shiffted_df
cleaned_df = append_frames(cleaned_df , frame)

In [9]:
for i in range(3):
    shiffted_df , columns , cols_merged = updatting_date(shiffted_df)
    shiffted_df = merge_function(shiffted_df , 2 , ['NAME ' , 'CATEGORY '])
    shiffted_df = shifting(shiffted_df , cols_merged , columns)
    frame = shiffted_df
    cleaned_df = append_frames(cleaned_df , frame)
    
cleaned_df = cleaned_df.drop(['shitted_col2','shitted_col1','shitted_col3'] , axis = 1)

C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3305884801.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'blank'] = df[list1[0]]
C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3305884801.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'blank'] = df[list1[0]]
C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3305884801.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

#### find duplicated values

In [10]:
cleaned_df['id'].duplicated().sum()

0

#### trim columns' header , make all countries' ISO upper case to avoid inconsistency

In [11]:
cleaned_df.columns = cleaned_df.columns.str.strip()
cleaned_df['country'] = cleaned_df['country'].apply(str.upper)

#### Check for shifted data

In [12]:
cleaned_df.groupby(['currency' , 'country'])['goal'].count()

currency      country
AUD           AU           6251
              NO            111
CAD           CA          12008
              NO            205
CHF           CH            471
              NO              7
DKK           DK            825
              NO             16
EUR           AT            377
              BE            402
              DE           2685
              ES           1373
              FR           1912
              IE            576
              IT           1751
              LU             40
              NL           2265
              NO            186
Film & Video  0               3
GBP           GB          27573
              NO            459
HKD           HK             97
MXN           MX            214
Music         0               4
NOK           NO            540
NZD           NO             28
              NZ           1137
SEK           NO             31
              SE           1271
SGD           SG            120
USD           NO  

In [13]:
sd = cleaned_df.copy()

shitted_data = sd.loc[sd['country']== '0' ]

cols_merged = shitted_data.columns.str.lower()

In [14]:
shitted_data = merge_function(shitted_data , 2 , ['name' , 'category'])
shitted_data.columns = shitted_data.columns.str.upper()
shitted_data.head(1)

C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3305884801.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'blank'] = df[list1[0]]


,ID,NAME,MAIN_CATEGORY,CURRENCY,DEADLINE,GOAL,LAUNCHED,PLEDGED,STATE,BACKERS,COUNTRY,USD PLEDGED
64484,1383759393,CD: Heartsong Harmonics Marc & Mary sing toge...,Music,Music,USD,11/3/2015 0:59,7085,9/14/2015 23:55,7470,undefined,0,"N,""0"


In [15]:
# cols_merged
shitted_data['blank'] = '-'

columns = shitted_data.columns

for i in range(len(cols_merged)):
    shitted_data = shitted_data.rename(columns = {columns[i] : cols_merged[i] })

shitted_data.head(1)

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
64484,1383759393,CD: Heartsong Harmonics Marc & Mary sing toge...,Music,Music,USD,11/3/2015 0:59,7085,9/14/2015 23:55,7470,undefined,0,"N,""0",-


In [16]:
sd = sd[sd['country'] != '0']
sd.shape

(323743, 13)

In [17]:
sd = pd.concat([sd, shitted_data])
sd.shape

(323750, 13)

In [18]:
sd.loc[sd['usd pledged']=='-', ['country', 'usd pledged']] = ['NO', 0]
cleaned_df = sd

#### find null values

In [19]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 323750 entries, 269970 to 177636
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             323750 non-null  int64 
 1   name           323746 non-null  object
 2   category       323750 non-null  object
 3   main_category  323750 non-null  object
 4   currency       323750 non-null  object
 5   deadline       323750 non-null  object
 6   goal           323750 non-null  object
 7   launched       323750 non-null  object
 8   pledged        323750 non-null  object
 9   state          323750 non-null  object
 10  backers        323750 non-null  object
 11  country        323750 non-null  object
 12  usd pledged    323750 non-null  object
dtypes: int64(1), object(12)
memory usage: 34.6+ MB


In [20]:
cleaned_df[cleaned_df['name'].isnull()]

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
142830,1848699072,NaN,Narrative Film,Film & Video,USD,2/29/2012 15:04,200000,1/1/2012 12:35,100,failed,1,US,100
262901,634871725,NaN,Video Games,Games,GBP,1/6/2013 23:00,2000,12/19/2012 23:57,196,failed,12,GB,317.7284362
265251,648853978,NaN,Product Design,Design,USD,7/18/2016 5:01,2500,6/18/2016 5:01,0,suspended,0,US,0
289847,796533179,NaN,Painting,Art,USD,12/5/2011 5:59,35000,11/6/2011 23:55,220,failed,5,US,220


In [21]:
cleaned_df = cleaned_df[cleaned_df['name'].notnull()]
cleaned_df.shape

(323746, 13)

In [22]:
cleaned_df['source'] = 'ks 2016'
cleaned_df.info()

C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\1998756166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['source'] = 'ks 2016'


<class 'pandas.core.frame.DataFrame'>
Index: 323746 entries, 269970 to 177636
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             323746 non-null  int64 
 1   name           323746 non-null  object
 2   category       323746 non-null  object
 3   main_category  323746 non-null  object
 4   currency       323746 non-null  object
 5   deadline       323746 non-null  object
 6   goal           323746 non-null  object
 7   launched       323746 non-null  object
 8   pledged        323746 non-null  object
 9   state          323746 non-null  object
 10  backers        323746 non-null  object
 11  country        323746 non-null  object
 12  usd pledged    323746 non-null  object
 13  source         323746 non-null  object
dtypes: int64(1), object(13)
memory usage: 37.0+ MB


In [23]:
cleaned_df[['goal' , 'pledged' , 'backers' , 'usd pledged']] = cleaned_df[['goal' , 'pledged' , 'backers' , 'usd pledged']].astype(float)

C:\Users\M .ATEF\AppData\Local\Temp\ipykernel_14176\3628077244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df[['goal' , 'pledged' , 'backers' , 'usd pledged']] = cleaned_df[['goal' , 'pledged' , 'backers' , 'usd pledged']].astype(float)


===================================================================================

# data of file 2018

In [26]:
ks_2018 = pd.read_csv(r"D:\mohamed\archive\ks-projects-201801)مشروع من النت).csv")
ks_2018.shape

(378661, 15)

#### find null values

In [27]:
ks_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                378661 non-null  int64  
 1   name              378657 non-null  object 
 2   category          378661 non-null  object 
 3   main_category     378661 non-null  object 
 4   currency          378661 non-null  object 
 5   deadline          378661 non-null  object 
 6   goal              378661 non-null  float64
 7   launched          378661 non-null  object 
 8   pledged           378661 non-null  float64
 9   state             378661 non-null  object 
 10  backers           378661 non-null  int64  
 11  country           378661 non-null  object 
 12  usd pledged       374864 non-null  float64
 13  usd_pledged_real  378661 non-null  float64
 14  usd_goal_real     378661 non-null  float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [28]:
ks_2018.loc[ks_2018['usd pledged'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
169,1000694855,STREETFIGHTERZ WHEELIE MURICA,Film & Video,Film & Video,USD,2014-09-20,6500.0,2014-08-06 21:28:36,555.00,undefined,0,"N,0""",NaN,555.00,6500.00
328,100149523,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,undefined,0,"N,0""",NaN,3402.08,3211.53
632,1003023003,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,undefined,0,"N,0""",NaN,3576.00,3500.00
647,1003130892,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,undefined,0,"N,0""",NaN,7007.80,6000.00
749,1003629045,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,undefined,0,"N,0""",NaN,3660.38,3000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378233,997971307,"EQUUS International Film Festival®, all-equine...",Film & Video,Film & Video,USD,2015-02-03,7500.0,2014-12-05 04:19:14,10.00,undefined,0,"N,0""",NaN,10.00,7500.00
378303,998319149,Emily 2050 - Short Film,Film & Video,Film & Video,CAD,2014-05-23,3000.0,2014-04-08 00:30:09,3102.00,undefined,0,"N,0""",NaN,2845.61,2752.04
378434,9988744,Matthew Stephens Music,Music,Music,USD,2016-02-05,5000.0,2016-01-06 21:59:23,235.00,undefined,0,"N,0""",NaN,235.00,5000.00
378585,999610349,Lady Vendredi: Afrofuturist concept 12 inch EP,Music,Music,GBP,2015-10-19,2000.0,2015-09-21 22:33:18,2125.00,undefined,0,"N,0""",NaN,3273.36,3080.81


In [29]:
ks_2018.loc[ks_2018['usd pledged'].isnull(), ['country', 'usd pledged']] = ['NO', 0]

ks_2018.groupby(['currency' , 'country'])['goal'].count()

currency  country
AUD       AU           7839
          NO            111
CAD       CA          14756
          NO            206
CHF       CH            761
          NO              7
DKK       DK           1113
          NO             16
EUR       AT            597
          BE            617
          DE           4171
          ES           2276
          FR           2939
          IE            811
          IT           2878
          LU             62
          NL           2868
          NO            186
GBP       GB          33672
          NO            460
HKD       HK            618
JPY       JP             40
MXN       MX           1752
NOK       NO            722
NZD       NO             28
          NZ           1447
SEK       NO             31
          SE           1757
SGD       SG            555
USD       NO           2738
          US         292627
Name: goal, dtype: int64

In [31]:
ks_2018.loc[ks_2018['name'].isnull()]

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
166851,1848699072,NaN,Narrative Film,Film & Video,USD,2012-02-29,200000.0,2012-01-01 12:35:31,100.0,failed,1,US,100.00,100.00,200000.00
307234,634871725,NaN,Video Games,Games,GBP,2013-01-06,2000.0,2012-12-19 23:57:48,196.0,failed,12,GB,317.73,316.05,3224.97
309991,648853978,NaN,Product Design,Design,USD,2016-07-18,2500.0,2016-06-18 05:01:47,0.0,suspended,0,US,0.00,0.00,2500.00
338931,796533179,NaN,Painting,Art,USD,2011-12-05,35000.0,2011-11-06 23:55:55,220.0,failed,5,US,220.00,220.00,35000.00


In [32]:
ks_2018 = ks_2018.dropna(axis = 0)

ks_2018['ID'].duplicated().sum()

0

In [33]:
ks_2018['source'] = 'ks 2018'

In [34]:
ks_2018 = ks_2018.rename(columns={'ID' : 'id'})
ks_2018 = ks_2018.drop(['usd_goal_real' , 'usd_pledged_real'] , axis = 1)

## concat the two files 

In [35]:
ks_2016 = cleaned_df

In [36]:
df = pd.concat([ks_2016 , ks_2018])

In [37]:
df.head(1)

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,source
269970,677103185,SixSixSeven Angels Demons Religion Esoteric,Graphic Novels,Comics,USD,10/10/2015 1:00,750.0,9/10/2015 18:15,25.0,failed,1.0,US,25.0,ks 2016


In [38]:
df['id'].duplicated().sum()

323746

In [39]:
df.drop_duplicates(subset=['id'] , inplace=True)

In [40]:
df.shape

(378657, 14)

In [41]:
df['deadline'] = pd.to_datetime(df['deadline'] , format='mixed')


df['launched'] = pd.to_datetime(df['launched'] , format='mixed')


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 378657 entries, 269970 to 378654
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   id             378657 non-null  int64         
 1   name           378657 non-null  object        
 2   category       378657 non-null  object        
 3   main_category  378657 non-null  object        
 4   currency       378657 non-null  object        
 5   deadline       378657 non-null  datetime64[ns]
 6   goal           378657 non-null  float64       
 7   launched       378657 non-null  datetime64[ns]
 8   pledged        378657 non-null  float64       
 9   state          378657 non-null  object        
 10  backers        378657 non-null  float64       
 11  country        378657 non-null  object        
 12  usd pledged    378657 non-null  float64       
 13  source         378657 non-null  object        
dtypes: datetime64[ns](2), float64(4), int64(1), object(7

#### #rows per file

In [43]:
df.groupby('source')['id'].count()

source
ks 2016    323746
ks 2018     54911
Name: id, dtype: int64

#### change wrong date

In [45]:
df.loc[(df['launched'].dt.year == 1970) & (df['country'] == 'US') , 'launched']= '1-1-2010'
df.loc[(df['launched'].dt.year == 1970) & (df['country'] == 'CH') , 'launched'] = '1-1-2015'

In [46]:
df.loc[df['launched'].dt.year == 1970]

,id,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,source


#### find outliers

In [52]:
import numpy as np

Q1 = np.quantile(df['goal'] , 0.25)
Q3 = np.quantile(df['goal'] , 0.75)
IQR = Q3 - Q1
outliers = df[(df['goal']<(Q1-1.5*IQR)) | (df['goal']>(Q3+1.5*IQR))]

print(f'outliters in range {Q1-1.5*IQR} & {Q3+1.5*IQR} for goal column')
# print(Q1-1.5*IQR , end='\n \n')
# print(Q3+1.5*IQR)

outliters in range -19000.0 & 37000.0 for goal column
